# Store Sales Average Model
---
Notebook ini merupakan studi mengenai hubungan antara Jenis Bangunan, Kelas Bangunan, Lokasi Kabupaten/Kota dan Luas Area Toko dalam men-generate penjualan per bulan di PT Prestasi Retail Innovation.
- Jenis Bangunan: Merupakan satu dari tiga tipe bangunan diantaranya Mall (M), Ruko (R) dan Bangunan Sendiri (S)
- Kelas Bangunan: Merupakan kelas dari tipe bangunan tersebut dalam men-generate sales. Misalkan Mall Grand Indonesia adalah Mall Kelas 1 (M1) sedangkan Gandaria City adalah Mall Kelas 4 (M4). Berikut adalah Kelas Bangunan yang ada.
  - Mall Kelas 1 (M1)
  - Mall Kelas 2 (M2)
  - Mall Kelas 3 (M3)
  - Mall Kelas 4 (M4)
  - Mall Kelas 5 (M5)
  - Ruko Kelas 1 (R1)
  - Ruko Kelas 2 (R2)
  - Ruko Kelas 3 (R3)
  - Ruko Kelas 4 (R4)
  - Ruko Kelas 5 (R5)
  - Bangunan Sendiri Kelas 1 (S1)
  - Bangunan Sendiri Kelas 2 (S2)
  - Bangunan Sendiri Kelas 3 (S3)
  - Bangunan Sendiri Kelas 4 (S4)
  - Bangunan Sendiri Kelas 5 (S5)
- Lokasi Kabupaten/Kota adalah lokasi geografis dari toko
- Luas Area Toko adalah luas meter persegi dari toko
---
Studi ini akan menggunakan Regresi Linear dalam merumuskan nilai Average Sales ($y$) yang dipengaruhi oleh variabel - variabel independen lainnya seperti Jenis Bangunan $x{_1}$, Kelas Bangunan $x{_2}$, Lokasi Kabupaten/Kota ($x{_3}$) dan Luas Area Toko ($x{_4}$). Berikut adalah formulasi Regresi Linear untuk permasalahan tersebut:  
  
$
y = ax{_1} + bx{_2} + cx{_3} + dx{_4} + e
$  
  
Dimana:  
$y$ = Prediksi Average Sales  
$x{_1}$ = Variabel independen mewakili Jenis Bangunan  
$x{_2}$ = Variabel independen mewakili Kelas Bangunan  
$x{_3}$ = Variabel independen mewakili Lokasi Kabupaten/Kota  
$x{_4}$ = Variabel independen mewakili Luas Area Toko  
$a$ = Koefisien variabel independen $x{_1}$  
$b$ = Koefisien variabel independen $x{_2}$  
$c$ = Koefisien variabel independen $x{_3}$  
$d$ = Koefisien variabel independen $x{_4}$  
$e$ = Bias dari Regresi Linear  

### Import Modul

In [ ]:
import random
random.seed(11)
from numpy.random import seed
seed(11)
from tensorflow.random import set_seed # type: ignore
set_seed(11)
import os
from typing import Literal
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from joblib import dump
import lightgbm
import xgboost
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
sns.set()

### Versi Tensorflow

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices())

### Format Tampilan DataFrame

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

## Eksplorasi Data
---

### Atribut Dasar Toko
Berikut adalah beberapa data atribut dasar toko yang saat ini dimiliki.

In [ ]:
data_toko = pd.read_excel("PRI - Store Renov Rent.xlsx", sheet_name=0, header=0)
data_toko

Karena kita tidak akan menggunakan semua kolom dalam data ini untuk kepentingan studi Store Sales Average, maka `data_toko` akan diringkas dan disusun ulang menjadi:

In [ ]:
data_toko = data_toko[["STORE CODE", "STORE NAME", "Tipe Bangunan", "Kelas Bangunan", "Kota Kabupaten 2", "Estimasi Populasi", "sqm"]]
data_toko

#### Luas Area Toko (sqm)

In [ ]:
data_luas = data_toko["sqm"]
hitung, bin = np.histogram(data_luas)
print(hitung, bin)

In [ ]:
plot = so.Plot(data_toko, "sqm")
plot.add(so.Bars(), so.Hist(), legend=True).label(title="Persebaran Toko berdasar SQM", x="Square Meters (sqm)", y="Jumlah Toko")

Berdasarkan luasnya (sqm), kita dapat melihat pada fungsi `histogram` di atas bahwa distribusi persebaran luas toko cukup normal dengan 29 Toko jatuh ke dalam kategori `sqm` di antara $75m{^2}$ sampai dengan $150m{^2}$, dengan 1 toko yang menjadi outlier dari distribusi dimana luas toko > $400m{^2}$.

#### Penjualan
---
Mengingat bahwa data *historical* yang dimiliki terbatas dari tahun 2018 sampai dengan November 2022, serta mengingat bahwa kita mengalami periode pandemi CoV-19 selama lebih dari 1 tahun, maka penulis merasa perlu untuk melakukan separasi data penjualan per bulan menggunakan flag `Pandemic`.
  
Berikut adalah sepenggal data penjualan *historical* per toko dari tahun 2018 sampai dengan November 2022 (40 baris data awal).

In [ ]:
data_penjualan = pd.read_excel("PRI - Store Renov Rent.xlsx", sheet_name="Sales", header=0)
data_penjualan

Berikut adalah grouping data penjualan *historical* yang disimpan dalam variabel `data_penjualan_by_month`.

In [ ]:
data_penjualan_by_month = data_penjualan.groupby(["EOM"]).sum(numeric_only=True)
data_penjualan_by_month

Untuk melihat trend pergerakan penjualan dalam kurun waktu ini, kita akan menggunakan `Simple Moving Average` yang akan menghitung rerata secara bergulung untuk interval waktu ke belakang (contoh: Moving Average 3 Bulan untuk Mei 2021 adalah rata - rata penjualan yang merupakan rata - rata dari penjualan di bulan Maret 2021, April 2021 dan Mei 2021).  
Namun satu `Moving Average` saja tidak dapat menggambarkan sebuah trend, karena itu kita juga akan menggunakan tambahan 2 interval `Moving Average` lainnya untuk menggambarkan trend pada jangka pendek, jangka menengah dan jangka panjang.  
Dalam studi ini kita akan menggunakan 3 `Simple Moving Average` yaitu:
* `MA3`: `Moving Average` dengan jendela periode 3 bulan ke belakang (Jangka Pendek)
* `MA6`: `Moving Average` dengan jendela periode 6 bulan ke belakang (Jangka Menengah)
* `MA12`: `Moving Average` dengan jendela periode 12 bulan atau 1 tahun ke belakang (Jangka Panjang)  
Berikut adalah `data_penjualan_by_month` dengan penambahan kolom `MA3`, `MA6` dan `MA12` yang didapat dengan menggunakan fungsi `rolling()` dari `pd.DataFrame` yang dirata-ratakan dengan fungsi `mean()`.

In [ ]:
data_penjualan_by_month["MA3"] = data_penjualan_by_month["Sales"].rolling(3).mean()
data_penjualan_by_month["MA6"] = data_penjualan_by_month["Sales"].rolling(6).mean()
data_penjualan_by_month["MA12"] = data_penjualan_by_month["Sales"].rolling(12).mean()
data_penjualan_by_month

Berikut adalah grafik penjualan *historical* dari tahun 2018 sampai dengan November 2022.

In [ ]:
garis_plot = so.Plot(data_penjualan_by_month, "EOM", "Sales")
garis_plot.add(so.Line()).label(title="Total Penjualan per Bulan (2018 - Nov 2022)", x="Tahun")

Berikut adalah grafik `Moving Average` untuk data penjualan *historical* dari tahun 2018 sampai dengan November 2022.

In [ ]:
print(data_penjualan_by_month.reset_index())

In [ ]:
data_kolom_melt = pd.melt(data_penjualan_by_month.reset_index().drop(columns=["Sales"]), id_vars='EOM', var_name="Tipe MA", value_name="Nilai") # type: ignore
data_kolom_melt

In [ ]:
# plt.plot(data_penjualan_by_month["MA3"], color="green", label="MA3")
# plt.plot(data_penjualan_by_month["MA6"], color="orange", label="MA6")
# plt.plot(data_penjualan_by_month["MA12"], color="red", label="MA12")
# plt.show()
fig, ax = plt.subplots()
ax.set_title("Moving Average Total Penjualan per Bulan")
ax.set_ylabel("Rata-Rata Total Penjualan per Bulan")
ax.set_xlabel("Tahun")
# Plot Moving Average
sns.lineplot(data_kolom_melt, x="EOM", y="Nilai", hue="Tipe MA")
# Region Section Pandemic
ax.fill_between(data_penjualan_by_month.index.values, 0, 28000000000, where=((data_penjualan_by_month.index.values > np.datetime64('2020-02-29')) & (data_penjualan_by_month.index.values <= np.datetime64('2021-10-31'))), color="red", alpha=0.2)
# Region Section Recovery
ax.fill_between(data_penjualan_by_month.index.values, 0, 28000000000, where=((data_penjualan_by_month.index.values >= np.datetime64('2021-11-01')) & (data_penjualan_by_month.index.values <= np.datetime64('2022-12-31'))), color="green", alpha=0.2)
plt.show()

Pada grafik di atas kita dapat melihat bahwa terdapat tren penurunan penjualan (terkonfirmasi dengan `MA3` yang turun ke bawah `MA6` dan `MA12`) pada periode Maret 2020 (`Pandemic`) dan nilai rata - rata penjualan ini bertahan cukup rendah hingga setidaknya sampai dengan bulan Oktober 2021 dan di bulan November 2021 hingga seterusnya kita dapat melihat nilai rata-rata penjualan per bulan yang meningkat (`Recovery`, terkonfirmasi dengan `MA3` yang melewati dan bertahan di atas `MA6` dan `MA12`).  
  
Oleh karena itu kita akan mengkategorikan penjualan yang terjadi diantara bulan Maret 2020 - Oktober 2021 sebagai penjualan dalam masa `Pandemic` dan lainnya sebagai penjualan `Normal`.

In [ ]:
pandemic_period = [
  np.datetime64('2020-03-31'),
  np.datetime64('2020-04-30'),
  np.datetime64('2020-05-31'),
  np.datetime64('2020-06-30'),
  np.datetime64('2020-07-31'),
  np.datetime64('2020-08-31'),
  np.datetime64('2020-09-30'),
  np.datetime64('2020-10-31'),
  np.datetime64('2020-11-30'),
  np.datetime64('2020-12-31'),
  np.datetime64('2021-01-31'),
  np.datetime64('2021-02-28'),
  np.datetime64('2021-03-31'),
  np.datetime64('2021-04-30'),
  np.datetime64('2021-05-31'),
  np.datetime64('2021-06-30'),
  np.datetime64('2021-07-31'),
  np.datetime64('2021-08-31'),
  np.datetime64('2021-09-30'),
  np.datetime64('2021-10-31'),
  ]
print(pandemic_period)

Berikut adalah pengkategorian bulan penjualan berdasarkan periode `Pandemic` dan `Normal`

In [ ]:
def status_pandemi(x):
  return "Pandemic" if x["EOM"] in pandemic_period else "Normal"

data_penjualan["Status Pandemi"] = data_penjualan.apply(lambda x: status_pandemi(x), axis=1)
data_penjualan

Pembentukan dataframe `data_penjualan_rerata` untuk lookup nilai penjualan rata-rata pada masa pandemi dan normal di dataframe `data_toko`

In [ ]:
data_penjualan_rerata = data_penjualan.groupby(['Status Pandemi', 'LocationCode']).mean(numeric_only=True)
data_penjualan_rerata

Implementasi lookup rata-rata penjualan per bulan untuk setiap toko baik pada masa pandemi maupun pada masa normal di dataframe `data_toko`

In [ ]:
data_rerata_penjualan = data_toko.copy()

def lookup_rerata(x, pandemi_status, lookup_df):
  try: 
    return sum(lookup_df.loc[pandemi_status, x['STORE CODE']].values)
  except:
    return np.NaN

data_rerata_penjualan["Rerata Penjualan Normal"] = data_rerata_penjualan.apply(lambda x: lookup_rerata(x, 'Normal', data_penjualan_rerata), axis=1) # type: ignore
data_rerata_penjualan["Rerata Penjualan Pandemi"] = data_rerata_penjualan.apply(lambda x: lookup_rerata(x, 'Pandemic', data_penjualan_rerata), axis=1) # type: ignore

data_rerata_penjualan

Pada dataframe `data_rerata_penjualan` dengan penambahan kolom `Rerata Penjualan Normal` dan `Rerata Penjualan Pandemi` kita dapat melihat bahwa nilai `Rerata Penjualan Normal` lebih besar daripada nilai `Rerata Penjualan Pandemi` untuk kesemua toko, hal ini menunjukkan bahwa kita berhasil menangkap nilai rata-rata penjualan per bulan di masa normal yang kita ekspektasikan menjadi acuan ke depannya.

Nilai pada kolom `Rerata Penjualan Normal` ini adalah nilai $y$ yang sebenarnya. Nilai $y$ yang sebenarnya ini akan menjadi acuan dalam proses pelatihan jaringan saraf tiruan untuk melihat seberapa akurat model dalam memprediksi nilai $y$ atau yang kita sebut $\hat{y}$ (*y-hat* atau prediksi y).

#### STORE CODE & KOTA KABUPATEN 2
---
Dalam membangun model prediksi, selain mempertimbangkan input dalam proses pelatihan model, kita juga harus mempertimbangkan interaksi pengguna dengan model nantinya dalam menghasilkan prediksi rata-rata penjualan per bulan.
Jika kita membayangkan pengguna melakukan input pada serangkaian form untuk mendapatkan nilai output prediksi rata-rata penjualan per bulan untuk input yang diberikan, nampaknya akan sulit jika pengguna menginput semisalkan `STORE CODE` 'FS040' atau `KOTA KABUPATEN 2` 'PALU'. Hal ini dikarenakan model akan dilatih menggunakan data pada `data_toko` yang jumlah sampelnya sangat terbatas dan tidak pernah mengenal 'FS040' atau 'PALU' sebagai salah satu input dalam proses pelatihan model.  
Oleh karena itu, kita akan melakukan modifikasi pada kedua variabel ini untuk memastikan proses pelatihan berjalan lebih umum (*general*) dan untuk memungkinkan input oleh pengguna pada model nantinya lebih umum.

##### STORE CODE
Untuk `STORE CODE`, supaya baik proses pelatihan maupun input pada model nantinya bisa berlaku secara lebih umum, kita akan menggunakan `SBU` yang diekstrak dari dua karakter pertama dalam `STORE CODE` dan untuk FO akan masuk ke dalam `SBU` 'Fisik Sport'

In [ ]:
data_store_code = data_rerata_penjualan.copy()

def konversi_sbu(x):
  try:
    match x['STORE CODE'][:2]:
      case "FS" | "FO":
        return "Fisik Sport"
      case "FF":
        return "Fisik Football"
      case "OD":
        return "Our Daily Dose"
      case _:
        return np.NaN
  except:
    return np.NaN

data_store_code["SBU"] = data_store_code.apply(lambda x: konversi_sbu(x), axis=1) # type: ignore

# Reorder kolom
kolom = ["STORE CODE", "STORE NAME", "SBU", "Tipe Bangunan", "Kelas Bangunan", "Kota Kabupaten 2", "Estimasi Populasi", "sqm", "Rerata Penjualan Normal", "Rerata Penjualan Pandemi"]
data_store_code = data_store_code[kolom]
  
data_store_code

##### KABUPATEN KOTA 2
Untuk `KABUPATEN KOTA 2`, kita akan melakukan grouping rentang populasi, misalkan populasi `0 - 500,000`, `500,001 - 1,000,000` dstnya. Hal ini dipandang lebih baik untuk proses pelatihan jaringan saraf tiruan model dan juga untuk implementasi prediksi model pada aplikasi ke depannya, mengingat jumlah sampel pelatihan yang sangat terbatas.  
Sebelumnya, dipandang perlu untuk melihat kardinalitas anggota dalam rentang yang terbentuk untuk memastikan distribusi yang mendekati normal.

In [ ]:
jumlah_anggota, bin = np.histogram(data_store_code["Estimasi Populasi"], bins=6, range=(0, 3000000))
print(f"Kardinalitas anggota: \t{jumlah_anggota}")
print(f"Range Bin: \t\t{bin}")

Pada fungsi `histogram()` di atas kita mengelompokkan data `Estimasi Populasi` ke dalam 6 rentang dengan nilai rentang minimal dimulai dari 0 dan nilai rentang maksimal sebesar 3,000,000.

In [ ]:
fig, ax = plt.subplots()

ax.set_title("Persebaran Rentang Populasi Toko")
ax.hist(bin[:-1], weights=jumlah_anggota, range=(0, 3000000))
ax.set_ylabel("Jumlah Toko di Kota dengan Rentang Populasi")
ax.set_xlabel("Rentang Populasi")

fig.show()

Pada grafik histogram di atas kita dapat melihat bahwa persebaran data cukup normal dimana sebagian besar toko dibuka di kota dengan populasi `1,000,000 - 1,500,000` (10 Toko) dan `1,500,001 - 2,000,000` (12 Toko) penduduk.

In [ ]:
data_rentang_populasi = data_store_code.copy()

def konversi_rentang_populasi(x):
  try:
    match x['Estimasi Populasi']:
      case x if x <= 500000:
        return '0 - 500000'
      case x if x <= 1000000:
        return '500001 - 1000000'
      case x if x <= 1500000:
        return '1000001 - 1500000'
      case x if x <= 2000000:
        return '1500001 - 2000000'
      case x if x <= 2500000:
        return '2000001 - 2500000'
      case _:
        return '> 2500000'
  except:
    return

# data_toko['Rentang Populasi'] = data_toko.apply(lambda x: konversi_rentang_populasi(x), axis=1) # type: ignore
data_rentang_populasi['Rentang Populasi'] = data_rentang_populasi.copy().apply(konversi_rentang_populasi, axis=1) # type: ignore

# Reorder kolom
kolom = ["STORE CODE", "STORE NAME", "SBU", "Tipe Bangunan", "Kelas Bangunan", "Kota Kabupaten 2", "Estimasi Populasi", "Rentang Populasi", "sqm", "Rerata Penjualan Normal", "Rerata Penjualan Pandemi"]
data_rentang_populasi = data_rentang_populasi[kolom]

data_rentang_populasi

#### Persebaran Data Categorical

In [ ]:
print(data_rentang_populasi["SBU"].value_counts())
data_rentang_populasi["SBU"].value_counts().plot(kind="bar", figsize=(3, 3), title="Persebaran Data SBU")

Persebaran data untuk jenis SBU masih bisa dianggap cukup normal dan tidak memiliki outlier maupun *skewness* yang signigikan.

In [ ]:
print(data_rentang_populasi["Kelas Bangunan"].value_counts())
data_rentang_populasi["Kelas Bangunan"].value_counts().sort_index().plot(kind="bar", figsize=(3, 3), title="Persebaran Data Kelas Bangunan")

Persebaran data untuk Kelas Bangunan seperti nampak diatas sekilas cukup normal. Namun jika kita hanya memperhitungkan Kelas Bangunan dalam kategori Mall, hanya ada 1 baris data yang masuk ke dalam kategori M5. Hal yang sama juga terjadi pada kategori Kelas Bangunan S5 yang hanya memiliki 1 anggota saja di dalam kategorinya. Untuk itu kita akan melakukan *feature engineering* untuk kolom SBU ini nantinya untuk mencapai distribusi data yang lebih normal dan merata dalam model.

In [ ]:
print(data_rentang_populasi["Rentang Populasi"].value_counts())
list_rentang_populasi = [
  "0 - 500000", 
  "500001 - 1000000", 
  "1000001 - 1500000", 
  "1500001 - 2000000", 
  "2000001 - 2500000", 
  "> 2500000"
]
data_rentang_populasi["Rentang Populasi"].value_counts().reindex(list_rentang_populasi).plot(kind="bar", figsize=(3,3), title="Persebaran Data Rentang Populasi")

Persebaran data untuk Rentang Populasi terlihat cukup normal meski memiliki kecenderungan (*skewness*) di sisi kanan.

#### Feature Engineering untuk Kelas Bangunan

Berdasarkan temuan pada bagian sebelumnya, kita akan melakukan *feature engineering* pada fitur Kelas Bangunan untuk mendapatkan distribusi kategori yang lebih normal dan kardinalitas anggota yang lebih baik untuk proses pelatihan nantinya.

Dengan asumsi sebagian besar pembukaan toko baru akan dilakukan di Mall dibandingkan dengan bangunan yang berdiri sendiri dan dengan mereduksi kategori Mall menjadi 3 kelas saja, maka kita akan melakukan mapping sebagai berikut:
- M1 tetap menjadi M1
- M2 dan M3 menjadi M2
- M4 tetap menjadi M3
- M5, R5 dan S5 akan menjadi M4NM (Mall 4/Non-Mall)

In [ ]:
data_prep = data_rentang_populasi.copy()

def feature_engineering_kb(x):
  match x["Kelas Bangunan"]:
    case "M1":
      return "M1"
    case "M2" | "M3":
      return "M2"
    case "M4":
      return "M3"
    case _:
      return "M4NM"

data_prep["Kelas Bangunan FE"] = data_prep.apply(lambda x: feature_engineering_kb(x), axis=1)

Persebaran data Kelas Bangunan FE

In [ ]:
data_prep["Kelas Bangunan FE"].value_counts()
data_prep["Kelas Bangunan FE"].value_counts().sort_index().plot(kind="bar", figsize=(3, 3), title="Persebaran Data Kelas Bangunan FE")

## Konversi Data Categorical ke dalam Label Encoder
---
Dengan perubahan-perubahan pada sub-bagian sebelumnya maka fungsi regresi linear dapat digambarkan ulang sebagai berikut:  
  
$
{Rerata Penjualan Normal} = a \cdot {SBU} + b \cdot {Kelas Bangunan FE} + c \cdot {Luas Area} + d \cdot {Rentang Populasi} + e
$ 
   
Dikarenakan `SBU`, `Kelas Bangunan FE` dan `Rentang Populasi` merupakan tipe data *categorical*, sedangkan pelatihan jaringan saraf tiruan untuk sebuah model memerlukan semua input dalam bentuk numerik, maka kita akan melakukan konversi pada ketiga data tersebut menjadi numerik.  
Dilihat dari jenis datanya, `SBU` dapat kita kategorikan sebagai *categorical nominal*, sedangkan `Rentang Populasi` dan `Kelas Kategori FE` merupakan *categorical ordinal*.  
Untuk data *categorical nominal* kita akan menerapkan proses *One Hot Encoding* untuk menerapakan pelabelan numerik tanpa susunan maupun bobot dan untuk data *categorical ordinal* kita akan menggunakan *Ordinal Encoding*.

In [ ]:
# One Hot Encoding SBU
ohe = OneHotEncoder(sparse_output=False)
sbu_encoded = ohe.fit_transform(data_prep["SBU"].values.reshape(-1, 1)) # type: ignore
data_sbu_encoded = pd.DataFrame(sbu_encoded, columns=ohe.get_feature_names_out(["SBU"]))
print(f"Data SBU setelah proses One Hot Encoding: \n{data_sbu_encoded}")

# Ordinal Encoding Kelas Bangunan dan Rentang Populasi
oe = OrdinalEncoder()
kelas_bangunan_fe_encoded = oe.fit_transform(data_prep["Kelas Bangunan FE"].values.reshape(-1, 1)) # type: ignore
data_kelas_bangunan_fe_encoded = pd.DataFrame(kelas_bangunan_fe_encoded, columns=["Kelas Bangunan FE Encoded"])
print(f"Data Kelas Bangunan FE setelah proses Ordinal Encoding: \n{data_kelas_bangunan_fe_encoded}")
oe = OrdinalEncoder(categories=[list_rentang_populasi])
rentang_populasi_encoded = oe.fit_transform(data_prep["Rentang Populasi"].values.reshape(-1, 1)) # type: ignore
data_rentang_populasi_encoded = pd.DataFrame(rentang_populasi_encoded, columns=["Rentang Populasi Encoded"])
print(f"Data Rentang Populasi setelah proses Ordinal Encoding: \n{data_rentang_populasi_encoded}")

Pemilihan fitur dan label untuk bahan pelatihan.

In [ ]:
data_model_full = pd.concat([data_prep, data_sbu_encoded, data_kelas_bangunan_fe_encoded, data_rentang_populasi_encoded], axis=1)
data_model_full

In [ ]:
data_final = data_model_full.copy()
data_final = data_final.drop(columns=["STORE CODE", "STORE NAME", "SBU", "Tipe Bangunan", "Kelas Bangunan", "Kelas Bangunan FE", "Kota Kabupaten 2", "Estimasi Populasi", "Rentang Populasi", "Rerata Penjualan Pandemi"], axis=1)
data_final = data_final[["sqm", "SBU_Fisik Football", "SBU_Fisik Sport", "SBU_Our Daily Dose", "Kelas Bangunan FE Encoded", "Rentang Populasi Encoded", "Rerata Penjualan Normal"]]
data_final.dropna(subset=["Rerata Penjualan Normal"], inplace=True)
data_final.reset_index(drop=True, inplace=True)
data_final

#### Eksplorasi Data Final

In [ ]:
data_final.describe()

In [ ]:
koefisien_korelasi = data_final.corr()
koefisien_korelasi

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(koefisien_korelasi, annot=True, cmap='magma')
plt.title("Heatmap Pearson Correlation Data Final")

Dan berikut adalah distribusi variabel independen terkait dengan variable independen lainnya.

In [ ]:
plot = sns.pairplot(data=data_final[["sqm", "Kelas Bangunan FE Encoded", "Rentang Populasi Encoded"]], diag_kind='kde')
plot.fig.suptitle("Distribusi antara Variabel Independen", y=1.02)

## Scaling dan Pembentukan Data Train Test
---

### Scaling
Merujuk kepada nilai dalam data yang dimiliki, kita sebenarnya hanya memiliki 1 fitur (kolom) dengan nilai numerik, yaitu kolom `sqm` sedangkan sisanya merupakan kategori yang di-encode baik secara One Hot Encoding maupun Label Encoding. Hal ini menyebabkan *mean* dari `sqm` memiliki nilai yang jauh berbeda dengan *mean* fitur - fitur lainnya, dan untuk mencegah proses update bobot dalam layer dari jaringan saraf tiruan dalam proses pelatihan memberikan bobot yang terlalu besar kepada `sqm` dibandingkan dengan fitur-fitur lainnya, kita akan menambahkan lapisan normalisasi untuk fitur `sqm`.

### Pembentukan Data Train Test

#### Data X y
Data X yang akan dipergunakan sebagai fitur adalah semua kolom pada `data_model` terkecuali kolom `Rerata Penjualan Normal` yang akan menjadi Data y.

In [ ]:
data_pelatihan = data_final.copy()

y = data_pelatihan["Rerata Penjualan Normal"]
X = data_pelatihan.drop("Rerata Penjualan Normal", axis=1)

print("Data X:")
print(X.to_string())
print("\nData y:")
print(y.to_string())

### Normalisasi dan Transformasi pada X_train dan X_test
Mengingat bahwa pada data `X_train` dan `X_test` kita hanya memiliki 1 fitur numerik (`sqm`) dan sisanya adalah *categorical encoding* maka skala antar fitur bisa terlihat sangat jomplang dan dapat menyebabkan model untuk membutuhkan waktu dalam dalam melakukan pemutakhiran bobot dalam proses *backpropagation* setiap epoch menggunakan optimizer-nya, maka dirasa perlu untuk melakukan normalisasi pada data `X_train` dan `X_test` untuk fitur `sqm`.
Serta dikarenakan jumlah sample yang sangat terbatas serta beberapa fitur *categorical encoded* yang memiliki kecondongan (*skewedness*) terhadap beberapa kategori saja, maka juga dirasa perlu untuk melakukan transformasi kuantil (menggunakan *quantile transformer*) pada data `X_train` dan `X_test`.

In [ ]:
X["sqm"].describe()

In [ ]:
# Normalisasi sqm pada Data X_train dan X_test
kolom_target_normalisasi = ["sqm"]
sc = StandardScaler()
X_norm = X.copy()
X_norm[kolom_target_normalisasi] = sc.fit_transform(X_norm[kolom_target_normalisasi]) # type: ignore

print ("Data X sqm setelah normalisasi:")
print(X_norm["sqm"]) # type: ignore

# Menyimpan normalisasi yang sudah di fit dengan X_train
dump(sc, 'normalizer/sqm_normalizer.bin', compress=True)

#### Data Train Test
Pembagian data train dan test adalah dengan rasio data test sebesar 0.3 dari total data, menggunakan random_state yang di-set pada 13

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=11, shuffle=True)
print(f"X_train:\n{X_train.to_string()}\n") # type: ignore
print(f"y_train:\n{y_train.to_string()}\n") # type: ignore
print(f"X_test:\n{X_test.to_string()}\n") # type: ignore
print(f"y_test:\n{y_test.to_string()}") # type: ignore

## Model
---

In [ ]:
# Plotting nilai sebenarnya dan prediksi
def plot_hasil(ax,
               prediksi,
               y_test,
               label_prediksi,
               label_y_test):
  ax.scatter(prediksi, y_test, c='crimson')
  ax.set_title("Mean Absolute Error")
  ax.set_yscale('log')
  ax.set_xscale('log')
  titik_awal = max(max(prediksi), max(y_test))
  titik_akhir = min(min(prediksi), min(y_test))
  ax.plot([titik_awal, titik_akhir], [titik_awal, titik_akhir], 'b-')
  ax.set_xlabel(label_prediksi)
  ax.set_ylabel(label_y_test)
  return ax

### Pembentukan Model
Pada bagian ini kita akan coba untuk membuat beberapa model yang akan dipergunakan dalam pelatihan nantinya. Pelatihan model akan dilakukan menggunakan modul regressor pada TensorFlow, Scikit, LGBM dan XGBoost.

#### TensorFlow
Beberapa model pelatihan jaringan saraf tiruan akan dilakukan menggunakan TensorFlow. Parameter dasar TensorFlow yang digunakan dalam kasus ini diantaranya:
- `loss` yang akan dimonitor adalah *mean absolute error* (terutama `val_mean_absolute_error` pada *callbacks*) pada model yang merupakan formula $\frac{\sum^n_{i=1}|\hat{y}-y_i|}{n}$ dimana $\hat{y}$ adalah nilai prediksi, $y_i$ adalah nilai sebenarnya dan $n$ adalah jumlah sampel dalam dataset validasi.
- `optimizer` yang digunakan dalam melakukan update bobot dan bias pada *neuron* dalam masing-masing *layer* dalam proses *backpropagation* di setiap *epoch* (atau dalam kasus ini pada setiap *batch*) adalah `Adam()` ([*Adaptive Moment Estimation*](https://arxiv.org/abs/1412.6980)) dengan *learning rate* 0.1.
- `metric` yang digunakan sama dengan `loss` yaitu `["mean_absolute_error"]`.
- Jumlah `epoch` untuk setiap model adalah 1000.
- Jumlah `batch_size` yang dipergunakan dalam proses `fit()` untuk mengupdate bobot dan bias pada *neuron* dalam masing-masing layer adalah 4.
- `callbacks` yang digunakan adalah:
  - `EarlyStopping()` dengan *patience* default 10, namun dalam studi kasus akan menggunakan *patience* 20 dan `monitor` yang diset untuk `val_mean_absolute_error`.
  - `ModelCheckpoint()` yang akan memonitor `val_mean_absolute_error` pada setiap *epoch* dan menyimpannya di folder `model/model.name`
- Terdapat setidaknya 2 model dasar, yaitu `model_dense_1_layer` dan `model_dnn_3_layer`. `model_dense_1_layer` hanya menggunakan 1 *dense layer* dengan fungsi aktivasi default `Linear`, sedangkan `model_dnn_3_layer` merupakan model dengan 3 *dense layer* yang terdiri dari 2 *dense layer* dengan fungsi aktivasi `ReLU` ([*Rectified Linear Unit*](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/)) dan jumlah *neuron* default 64 (dapat disetting berbeda) serta 1 *dense layer* (output) dengan fungsi aktivasi `Linear`.

In [ ]:


## TENSORFLOW MODEL
class TensorFlow:
  # Inisiasi kelas dan parameter model
  def __init__(self, 
               es_patience: int = 0,
               callbacks: list = [],
               loss: Literal["mae", "mse"] = "mae",
               optimizer: Literal["adam", "sgd", "adadelta"] = "adam",
               optimizer_lr: float = 0.1,
               metric: list = ['mean_absolute_error'],
               epoch: int = 100,
               verbose_mode: int = 0):
    self.es_patience = es_patience
    self.callbacks = callbacks
    self.optimizer_lr = optimizer_lr
    self.metric = metric
    self.epoch = epoch
    match loss:
      case "mse":
        self.loss = tf.keras.losses.mse
      case _:
        self.loss = tf.keras.losses.mae
    match optimizer:
      case "sgd":
        self.optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=self.optimizer_lr)
      case "adadelta":
        self.optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=self.optimizer_lr)
      case _:
        self.optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=self.optimizer_lr)
    self.verbose_mode = verbose_mode
    
  # Setting callbacks
  def callbacks_model(self, model_compiled):
    if len(self.callbacks) == 0:
      return [tf.keras.callbacks.EarlyStopping(monitor="val_mean_absolute_error", 
                                              patience=10 if self.es_patience <= 0 else self.es_patience,
                                              restore_best_weights=True,
                                              verbose=0),
              tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(os.getcwd(), f"model/{model_compiled.name}"),
                                                monitor="val_mean_absolute_error",
                                                save_best_only=True,
                                                verbose=0)]
    else:
      return self.callbacks
          
  # Compile model
  def compile_model(self, model):
    model.compile(loss=self.loss,
                  optimizer=self.optimizer,
                  metrics=self.metric)
    
  # Fit model
  def fit_model(self, X_train, y_train, X_test, y_test, model):
    return model.fit(X_train, 
                     y_train, 
                     epochs=self.epoch, 
                     validation_data=(X_test, y_test), 
                     callbacks=self.callbacks_model(model), 
                     verbose=self.verbose_mode,
                     batch_size=4)
  
  # Model_Regresi_linear_1_Layer
  def model_dense_1_layer(self,
                          nama_model: str = ""):
    # Model Def
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(1, input_dim=X_train.shape[1]) # type: ignore
    ], name=nama_model)
    self.compile_model(model)
    return model
  
  # Model_Regresi_Linear_3_Layer_2_RELU
  def model_dnn_3_layer(self, 
                        unit_1: int = 64, 
                        unit_2: int = 64,
                        nama_model: str = ""):
    # Model Def
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(unit_1, activation="relu", name=f"dense_layer_1_{unit_1}_nodes_relu_activation"),
      tf.keras.layers.Dense(unit_2, activation="relu", name=f"dense_layer_2_{unit_2}_nodes_relu_activation"),
      tf.keras.layers.Dense(1, name="dense_layer_output_1_node_linear_activation")
    ], name=nama_model)
    self.compile_model(model)
    return model

#### TensorFlow dengan Cross Validation
Mengingat sampel yang sangat terbatas jumlahnya dalam dataset, kita juga akan mencoba untuk melakukan teknik [*cross-validation*](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) untuk menangani jumlah sampel yang amat sedikit ini. Jenis *cross-validation* yang akan digunakan dalam pelatihan model adalah *K-Fold Cross Validation* dan *Leave-One-Out Cross Validation*.

In [ ]:
class TensorFlowCV():
  def __init__(self, X, y):
    self.X = X
    self.y = y
  
  def callback(self, teknik_fold: str, fold_ke: int, model: tf.keras.Model, es_patience: int = 10):
    return [
      tf.keras.callbacks.EarlyStopping(monitor="val_mean_absolute_error", 
                                       patience=es_patience,
                                       restore_best_weights=True,
                                       verbose=0),
      tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(os.getcwd(), f"model/{teknik_fold}_n_{fold_ke}_{model.name}"),
                                         monitor="val_mean_absolute_error",
                                         save_best_only=True,
                                         verbose=0)]
    
  def fit_model(self, model, X, y, validation_data: tuple, epoch: int, callbacks: list, verbose: Literal[0, 1, 2]):
    hasil = model.fit(x=X,
                      y=y,
                      epochs=epoch,
                      validation_data=validation_data,
                      callbacks=callbacks,
                      verbose=verbose)
    return hasil
  
  def ekstrak_hasil(self, output_training, prefix_judul):
    # List parameter model
    output_param_model = []

    # Loop output_training
    if len(output_training) != 0:
      for index, output in enumerate(output_training):    
        # newline
        if index != 0:
          print("\n")
        # y_test
        if "y_test" in output:
          y_test = output["y_test"]
        # Subplot
        fig, ax = plt.subplots(1, 2)
        fig.set_figwidth(20)
        fig.set_figheight(3)
        fig.suptitle(f"{prefix_judul}_n_{index+1}_{output['model'].name}")
        # Subplot 1 Kurva Loss
        pd.DataFrame(output['hasil'].history).plot(ax=ax[0])
        ax[0].set_title("Kurva Loss")
        ax[0].set_xlabel('Epochs')
        ax[0].set_ylabel('Loss')
        # Subplot 2 Mean Absolute Error pada Validation Dataset
        plot_hasil(ax[1], 
                  output['prediksi'],
                  y_test.tolist(), # type: ignore
                  "Rerata Penjualan Normal\n(prediksi)",
                  "Rerata Penjualan Normal\n(nilai sebenarnya)")
        plt.show()
        # Struktur model
        output['model'].summary()
        # Parameter output model
        r2 = r2_score(y_test.tolist(), output['prediksi']) # type: ignore
        mae = output['evaluasi']['mean_absolute_error']
        mse = mean_squared_error(y_test.tolist(), output['prediksi']) # type: ignore
        # Print parameter model
        print(f"Skor R2: {'{:,.2%}'.format(r2) if r2 is not None else None}")
        print(f"Mean Absolute Error: {'{:,.0f}'.format(mae)}")
        print(f"Mean Squared Error: {'{:,.0f}'.format(mse)}")
        # Simpan parameter output model dalam dictionary
        dict_param_model = {
          "Nama Model": f"{(prefix_judul + '_n_' + str(index+1) +'_') if prefix_judul is not None else ''}{output['model'].name}",
          "Skor R2": r2,
          "Mean Absolute Error": mae,
          "Mean Squared Error": mse
        }
        output_param_model.append(dict_param_model)
        
      print(output_param_model) 
      return output_param_model
  
  def kfold_cv(self, model: tf.keras.Model, jumlah_split: int = 13, epoch: int = 10, es_patience: int = 10):
    # K-Fold split
    kfold = KFold(n_splits=jumlah_split, shuffle=True)
    # list untuk menyimpan hasil pada setiap fold
    list_output_training_kfold = []
    
    # Loop dalam fold
    for fold, (train_indeks, test_indeks) in enumerate(kfold.split(self.X, self.y)):
      print(f"Fold ke {fold + 1} dari {jumlah_split}:")
      # Train dan test untuk fold ini
      X_train, y_train = self.X.iloc[train_indeks], self.y.iloc[train_indeks]
      X_test, y_test = self.X.iloc[test_indeks], self.y.iloc[test_indeks]
      # Fit model
      hasil = self.fit_model(
        model=model, 
        X=X_train, 
        y=y_train, 
        validation_data=(X_test, y_test), 
        epoch=epoch, 
        callbacks=self.callback(teknik_fold="kfold", 
                                fold_ke=fold + 1, 
                                model=model, 
                                es_patience=es_patience), 
        verbose=0)
      # Output training
      evaluasi = model.evaluate(X_test, y_test, verbose=0, return_dict=True) # type: ignore
      prediksi = list(np.concatenate(model.predict(X_test, verbose=0)).flat) # type: ignore
      dict_output_model = {
        "model": model,
        "hasil": hasil,
        "evaluasi": evaluasi,
        "prediksi": prediksi,
        "y_test": y_test
      }
      print(dict_output_model)
      list_output_training_kfold.append(dict_output_model)
    
    # Output model
    output_kfold = self.ekstrak_hasil(list_output_training_kfold, "kfold")
    
    # return model fit
    return output_kfold, list_output_training_kfold
    
  def loo_cv(self, model: tf.keras.Model, epoch: int = 10, es_patience: int = 10):
    # LOOCV
    loo = LeaveOneOut()
    # list untuk menyimpan hasil pada setiap fold
    list_output_training_loo = []
    
    # Loop dalam fold
    for fold, (train_indeks, test_indeks) in enumerate(loo.split(self.X)):
      print(f"Fold ke {fold + 1} dari {len(X)}:")
      # Train dan test untuk fold ini
      X_train, y_train = self.X.iloc[train_indeks], self.y.iloc[train_indeks]
      X_test, y_test = self.X.iloc[test_indeks], self.y.iloc[test_indeks]
      # Fit model
      hasil = self.fit_model(
        model=model,
        X=X_train,
        y=y_train,
        validation_data=(X_test, y_test),
        epoch=epoch,
        callbacks=self.callback(teknik_fold="leaveoneout",
                                fold_ke=fold + 1,
                                model=model,
                                es_patience=es_patience),
        verbose=0
      )
      # Output training
      evaluasi = model.evaluate(X_test, y_test, verbose=0, return_dict=True) # type: ignore
      prediksi = list(np.concatenate(model.predict(X_test, verbose=0)).flat) # type: ignore
      dict_output_model = {
        "model": model,
        "hasil": hasil,
        "evaluasi": evaluasi,
        "prediksi": prediksi,
        "y_test": y_test
      }
      print(dict_output_model)
      list_output_training_loo.append(dict_output_model)
      
    # output model
    output_loo = self.ekstrak_hasil(list_output_training_loo, "leaveoneout")
    
    # return model fit
    return output_loo, list_output_training_loo

#### Scikit
Fungsi regresi linear yang akan dipergunakan dari modul Scikit adalah `LinearRegression()` dan `DecisionTreeRegressor()`. Tidak banyak kustomisasi yang dapat dilakukan pada modul ini karena sifat *built-in* dari fungsi-fungsi tersebut secara default dinilai sudah cukup baik.

In [ ]:
## SCIKIT MODEL
class SK:
  # Inisiasi kelas dan parameter model
  def __init__(self, 
               fitur_train: pd.DataFrame, 
               target_train: pd.DataFrame,
               fitur_test: pd.DataFrame,
               target_test: pd.DataFrame):
    self.fitur_train = fitur_train
    self.target_train = target_train
    self.fitur_test = fitur_test
    self.target_test = target_test
    
  # Fit model
  def fit_model(self, model):
    return model.fit(self.fitur_train, self.target_train)
  
  # Plotting nilai sebenarnya dan prediksi
  def plot_hasil(self):
    return
  
  # Model_SK_Linear_Regresi
  def regresi_linear(self):
    return LinearRegression()
  
  # Model_SK_Decision_Tree
  def decision_tree(self):
    return DecisionTreeRegressor()
  
  # Model_SK_Random_Forest
  def random_forest(self):
    return RandomForestRegressor()
  
  # Model_SK_Gradient_Boosting
  def gradient_boosting(self):
    return GradientBoostingRegressor()

#### LightGBM dan XGBoost

Dua pustaka pembelajaran mesin lainnya yang akan digunakan diantaranya *light gradient-boosting machine* ([LightGBM](https://lightgbm.readthedocs.io))dan *extreme gradient boosting* ([XGBoost](https://xgboost.readthedocs.io)). Sama dengan Scikit, tidak banyak kustomisasi yang dilakukan untuk parameter di dalam dua pustaka ini.

In [ ]:
# LIGHTGBM DAN XGBOOST MODEL (ENSEMBLE REGRESSOR)
class EnsembleModel:
  def __init__(self,
               fitur_train: pd.DataFrame,
               target_train: pd.DataFrame,
               fitur_test: pd.DataFrame,
               target_test: pd.DataFrame):
    self.fitur_train = fitur_train
    self.target_train = target_train
    self.fitur_test = fitur_test
    self.target_test = target_test
  
  def fit_model(self, model):
    return model.fit(self.fitur_train, self.target_train)
  
  def lgbm(self):
    return lightgbm.LGBMRegressor()
  
  def xgb(self):
    return xgboost.XGBRFRegressor()

### Pelatihan Model

#### TensorFlow
Kita akan melakukan pelatihan jaringan saraf tiruan menggunakan TensorFlow untuk setidaknya 7 model, yaitu:
- Model DNN 3 Layer ReLU 16 16 (`Model_DNN_3_Layer_RELU_16_16`) dengan jumlah *neuron* pada *dense layer* pertama dan kedua masing-masing 16 *neuron*.
- Model DNN 3 Layer ReLU 32 16 (`Model_DNN_3_Layer_RELU_32_16`) dengan jumlah *neuron* pada *dense layer* pertama dan kedua masing-masing 32 dan 16 *neuron* secara berturut-turut.
- Model DNN 3 Layer ReLU 32 32 (`Model_DNN_3_Layer_RELU_32_32`) dengan jumlah *neuron* pada *dense layer* pertama dan kedua masing-masing 32 *neuron*.
- Model DNN 3 Layer ReLU 64 32 (`Model_DNN_3_Layer_RELU_64_32`) dengan jumlah *neuron* pada *dense layer* pertama dan kedua masing-masing 64 dan 32 *neuron* secara berturut-turut.
- Model DNN 3 Layer ReLU 64 64 (`Model_DNN_3_Layer_RELU_64_64`) dengan jumlah *neuron* pada *dense layer* pertama dan kedua masing-masing 64 *neuron* atau nilai default `unit_size` pada `model_dnn_3_layer`.
- Model DNN 3 Layer ReLU 128 64 (`Model_DNN_3_Layer_RELU_128_64`) dengan jumlah *neuron* pada *dense layer* pertama dan kedua masing-masing 128 dan 64 *neuron* secara berturut-turut.
- Model DNN 3 Layer ReLU 128 128 (`Model_DNN_3_Layer_RELU_128_128`) dengan jumlah *neuron* pada *dense layer* pertama dan kedua masing-masing 128 *neuron*.  
  
Parameter `patience` dari `tf.keras.callbacks.EarlyStopping()` di-set menjadi 20 *epoch* dan jumlah `epoch` yang digunakan dalam setiap pelatihan adalah 1000.

In [ ]:
tensorflow = TensorFlow(
  es_patience=20,
  epoch=1000
)

# List model yang di-compile
tensorflow_model = [tensorflow.model_dnn_3_layer(16, 16, "Model_DNN_3_Layer_RELU_16_16"),
                    tensorflow.model_dnn_3_layer(32, 16, "Model_DNN_3_Layer_RELU_32_16"),
                    tensorflow.model_dnn_3_layer(32, 32, "Model_DNN_3_Layer_RELU_32_32"), 
                    tensorflow.model_dnn_3_layer(64, 32, "Model_DNN_3_Layer_RELU_64_32"), 
                    tensorflow.model_dnn_3_layer(nama_model="Model_DNN_3_Layer_RELU_64_64"), 
                    tensorflow.model_dnn_3_layer(128, 64, "Model_DNN_3_Layer_RELU_128_64"), 
                    tensorflow.model_dnn_3_layer(128, 128, "Model_DNN_3_Layer_RELU_128_128")]

# List output pelatihan model tensorflow
output_training = []

# Loop fit dan simpan model
for index, model in enumerate(tensorflow_model):
  # Fit setiap model dan simpan di variabel hasil
  hasil = tensorflow.fit_model(X_train, y_train, X_test, y_test, model)
  # Simpan evaluasi model
  evaluasi = model.evaluate(X_test, y_test, verbose=0, return_dict=True) # type: ignore
  # Lakukan prediksi model
  prediksi = list(np.concatenate(model.predict(X_test, verbose=0)).flat) # type: ignore
  dict_output_model = {
    "model": model,
    "hasil": hasil,
    "evaluasi": evaluasi,
    "prediksi": prediksi
    }
  print(dict_output_model)
  output_training.append(dict_output_model)
  
# List parameter model
output_param_model = []

# Loop output_training
if len(output_training) != 0:
  for index, output in enumerate(output_training):    
    # newline
    if index != 0:
      print("\n")
    # Subplot
    fig, ax = plt.subplots(1, 2)
    fig.set_figwidth(20)
    fig.set_figheight(3)
    fig.suptitle(f"{output['model'].name}")
    # Subplot 1 Kurva Loss
    pd.DataFrame(output['hasil'].history).plot(ax=ax[0])
    ax[0].set_title("Kurva Loss")
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')
    # Subplot 2 Mean Absolute Error pada Validation Dataset
    plot_hasil(ax[1], 
              output['prediksi'],
              y_test.tolist(), # type: ignore
              "Rerata Penjualan Normal\n(prediksi)",
              "Rerata Penjualan Normal\n(nilai sebenarnya)")
    plt.show()
    # Struktur model
    output['model'].summary()
    # Parameter output model
    r2 = r2_score(y_test.tolist(), output['prediksi']) # type: ignore
    mae = output['evaluasi']['mean_absolute_error']
    mse = mean_squared_error(y_test.tolist(), output['prediksi']) # type: ignore
    # Print parameter model
    print(f"Skor R2: {'{:,.2%}'.format(r2)}")
    print(f"Mean Absolute Error: {'{:,.0f}'.format(mae)}")
    print(f"Mean Squared Error: {'{:,.0f}'.format(mse)}")
    # Simpan parameter output model dalam dictionary
    dict_param_model = {
      "Nama Model": output['model'].name,
      "Skor R2": r2,
      "Mean Absolute Error": mae,
      "Mean Squared Error": mse
    }
    output_param_model.append(dict_param_model)

#### Ringkasan Pelatihan Model TensorFlow

In [ ]:
# Buat nama kolom
list_nama_kolom = ['Skor R2', 'Mean Absolute Error', 'Mean Squared Error']
# Ekstrak nama model dari output_param_model
list_nama_model = [model['Nama Model'] for model in output_param_model]
# Ekstrak nilai output param masing - masing model
# Loop berdasar nama model
list_parameter_model = []
for indeks in range(len(list_nama_model)):
  # ambil r2, mae dan mse
  list_parameter_model.append([
    output_param_model[indeks]['Skor R2'],
    output_param_model[indeks]['Mean Absolute Error'],
    output_param_model[indeks]['Mean Squared Error']
    ])
# Buat dataframe ringkasan
df_ringkasan = pd.DataFrame(list_parameter_model, 
                             columns=list_nama_kolom, 
                             index=list_nama_model)

# Print model terbaik untuk masing - masing kategori
r2_terbaik, mae_terbaik, mse_terbaik = (
  df_ringkasan[list_nama_kolom[0]].idxmax(),
  df_ringkasan[list_nama_kolom[1]].idxmin(),
  df_ringkasan[list_nama_kolom[2]].idxmin()
  )
print(f"Model dengan {list_nama_kolom[0]} terbaik:\t\t\t\t{r2_terbaik}")
print(f"Model dengan {list_nama_kolom[1]} terbaik:\t\t{mae_terbaik}")
print(f"Model dengan {list_nama_kolom[2]} terbaik:\t\t{mse_terbaik}\n")

# Formatting dataframe dan tampilkan df_ringkasan
df_ringkasan[list_nama_kolom[0]] = df_ringkasan[list_nama_kolom[0]].map('{:,.2%}'.format)
df_ringkasan[list_nama_kolom[1]] = df_ringkasan[list_nama_kolom[1]].map('{:,.0f}'.format)
df_ringkasan[list_nama_kolom[2]] = df_ringkasan[list_nama_kolom[2]].map('{:,.0f}'.format)
print(df_ringkasan.to_string())

#### TensorFlow dengan Cross Validation
Pada bagian ini kita akan melakukan pelatihan lebih lanjut untuk model terbaik dari pelatihan tensorflow sebelumnya. Kali ini kita akan menggunakan teknik *cross validation* untuk mengatasi jumlah sampel dalam dataset yang sangat terbatas.

In [ ]:
# Memilih model tensorflow terbaik sebelumnya
indeks_model_terbaik = None
for indeks, baris in enumerate(df_ringkasan.index):
  if baris == mae_terbaik:
    indeks_model_terbaik = indeks

#### K-Fold Cross Validation pada Model Terbaik

In [ ]:
if indeks_model_terbaik is not None:
  tensorflow_cv = TensorFlowCV(X_norm, y)
  model_terbaik = tensorflow_model[indeks_model_terbaik]
  output_param_kfold, hasil_kfold = tensorflow_cv.kfold_cv(model=model_terbaik,
                                                           epoch=1000,
                                                           es_patience=10)

#### Leave-One-Out Cross Validation pada Model Terbaik

In [ ]:
if indeks_model_terbaik is not None:
  output_param_loo, hasil_loo = tensorflow_cv.loo_cv(model=model_terbaik,
                                                     epoch=1000,
                                                     es_patience=10)

#### Scikit

In [ ]:
sk = SK(fitur_train=X_train, # type: ignore
        target_train=y_train, # type: ignore
        fitur_test=X_test, # type: ignore
        target_test=y_test) # type: ignore
sk_model = [sk.regresi_linear(), sk.decision_tree(), sk.random_forest(), sk.gradient_boosting()]
# Nama Model
list_nama_model = [
  'Linear Regression',
  'Decision Tree Regressor',
  'Random Forest Regressor',
  'Gradient Boosting Regressor'
  ]
for index, model in enumerate(sk_model):
  hasil = sk.fit_model(model)
  prediksi = hasil.predict(X_test)
  r2 = r2_score(y_test, prediksi)
  mae = mean_absolute_error(y_test, prediksi)
  mse = mean_squared_error(y_test, prediksi)
  fig, ax = plt.subplots()
  plot_hasil(ax,
             prediksi,
             y_test.tolist(), # type: ignore
             "Rerata Penjualan Normal\n(prediksi)",
             "Rerata Penjualan Normal\n(nilai sebenarnya)")
  fig.suptitle(list_nama_model[index])
  plt.show()
  print(f"Skor R2: {'{:,.2%}'.format(r2)}")
  print(f"Mean Absolute Error: {'{:,.0f}'.format(mae)}")
  print(f"Mean Squared Error: {'{:,.0f}'.format(mse)}")
  # Simpan parameter output model dalam dictionary
  dict_param_model = {
    "Nama Model": list_nama_model[index],
    "Skor R2": r2,
    "Mean Absolute Error": mae,
    "Mean Squared Error": mse
  }
  output_param_model.append(dict_param_model)

#### LightGBM dan XGBoost

In [ ]:
ensemble = EnsembleModel(fitur_train=X_train, # type: ignore
                         target_train=y_train, # type: ignore
                         fitur_test=X_test, # type: ignore
                         target_test=y_test) # type: ignore
ensemble_model = [ensemble.lgbm(), ensemble.xgb()]
# Nama Model
list_nama_model = [
  'LightGBM',
  'XGBoost'
  ]
for index, model in enumerate(ensemble_model):
  hasil = ensemble.fit_model(model)
  prediksi = hasil.predict(X_test)
  r2 = r2_score(y_test, prediksi)
  mae = mean_absolute_error(y_test, prediksi)
  mse = mean_squared_error(y_test, prediksi)
  fig, ax = plt.subplots()
  plot_hasil(ax,
             prediksi,
             y_test.tolist(), # type: ignore
             "Rerata Penjualan Normal\n(prediksi)",
             "Rerata Penjualan Normal\n(nilai sebenarnya)")
  fig.suptitle(list_nama_model[index])
  plt.show()
  print(f"Skor R2: {'{:,.2%}'.format(r2)}")
  print(f"Mean Absolute Error: {'{:,.0f}'.format(mae)}")
  print(f"Mean Squared Error: {'{:,.0f}'.format(mse)}")
  # Simpan parameter output model dalam dictionary
  dict_param_model = {
    "Nama Model": "LightGBM" if index == 0 else "XGBoost",
    "Skor R2": r2,
    "Mean Absolute Error": mae,
    "Mean Squared Error": mse
  }
  output_param_model.append(dict_param_model)

## Kesimpulan

In [ ]:
# Buat nama kolom
list_nama_kolom = ['Skor R2', 'Mean Absolute Error', 'Mean Squared Error']
# list output param model
list_out_param_model = [output_param_model, output_param_kfold, output_param_loo]
# Ekstrak nama model dan parameter model dari list_out_param_model
list_nama_model = []
list_parameter_model = []
for list_param in list_out_param_model:
  for model in list_param:
    list_nama_model.append(model['Nama Model'])
    list_parameter_model.append([
      model['Skor R2'],
      model['Mean Absolute Error'],
      model['Mean Squared Error']
    ])
# Buat dataframe kesimpulan
df_kesimpulan = pd.DataFrame(list_parameter_model,
                             columns=list_nama_kolom,
                             index=list_nama_model)
# Print model terbaik untuk masing - masing kategori
r2_terbaik, mae_terbaik, mse_terbaik = (
  df_kesimpulan[list_nama_kolom[0]].idxmax(),
  df_kesimpulan[list_nama_kolom[1]].idxmin(),
  df_kesimpulan[list_nama_kolom[2]].idxmin()
  )
print(f"Model dengan {list_nama_kolom[0]} terbaik:\t\t\t\t{r2_terbaik}")
print(f"Model dengan {list_nama_kolom[1]} terbaik:\t\t{mae_terbaik}")
print(f"Model dengan {list_nama_kolom[2]} terbaik:\t\t{mse_terbaik}\n")

# Formatting dataframe dan tampilkan df_kesimpulan
df_kesimpulan[list_nama_kolom[0]] = df_kesimpulan[list_nama_kolom[0]].map('{:,.2%}'.format)
df_kesimpulan[list_nama_kolom[1]] = df_kesimpulan[list_nama_kolom[1]].map('{:,.0f}'.format)
df_kesimpulan[list_nama_kolom[2]] = df_kesimpulan[list_nama_kolom[2]].map('{:,.0f}'.format)
print(df_kesimpulan.to_string())

#### Evaluasi Model Terbaik pada `y_test`

In [ ]:
def evaluasi_model(nama_model: str):
  print(f"NAMA MODEL: {nama_model}")
  loaded_model = tf.saved_model.load(os.path.join(os.getcwd(), f"model/{nama_model}"))
  list_prediksi = []
  list_baris = []
  for x in X_test.iterrows(): # type: ignore
    list_nilai = []
    for row in x[1]:
      list_nilai.append(row)
    list_baris.append(list_nilai)
  prediksi = list(np.concatenate(loaded_model(list_baris)).flat) # type: ignore
  list_prediksi.append(prediksi)
  list_prediksi_final = []
  for nilai in list_prediksi[0]:
    list_prediksi_final.append([nilai])
  df_evaluasi = pd.DataFrame(list_prediksi_final, columns=['Rerata Penjualan Normal (prediksi)'])
  y_test.reset_index(drop=True, inplace=True) # type: ignore
  df_evaluasi_final = pd.concat([y_test, df_evaluasi], axis=1) # type: ignore
  df_evaluasi_final["error"] = df_evaluasi_final["Rerata Penjualan Normal (prediksi)"] - df_evaluasi_final["Rerata Penjualan Normal"]
  print(df_evaluasi_final.to_string())
  print("")
  print("Mean Absolute Error pada model dengan dataset y_test:")
  print('{:,.0f}'.format(abs(df_evaluasi_final["error"]).mean()))
  
list_model_terbaik = [r2_terbaik, mae_terbaik]
for indeks, model in enumerate(list_model_terbaik):
  if indeks != 0:
    print("")
  evaluasi_model(model) # type: ignore

Berdasarkan hasil akhir dari setiap model yang sudah melalu proses pelatihan dalam jaringan saraf tiruan diatas, kita dapat menyimpulkan bahwa `Model_DNN_3_Layer_RELU_128_128` pada *cross validation* `LeaveOneOut` n ke 30 (`leaveoneout_n_30_Model_DNN_3_Layer_RELU_128_128`) adalah model dengan performa terbaik diantara model-model dalam pelatihan jaringan saraf tiruan untuk semua kategori parameter evaluasi MAE.

Untuk implementasi model ini ke dalam aplikasi ke depannya, kita akan melakukan konversi model terbaik dari format `protobuf (.pb)` menjadi `tensorflow lite (.tlite)` untuk disisipkan pada aplikasi dan melakukan inferensi terhadap input dari user.

In [ ]:
# Konverter
konverter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(os.getcwd(), f"model/{mae_terbaik}"))
konverter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS] # type: ignore
konverter.allow_custom_ops = True
konverter.experimental_new_converter = True
tflite_model = konverter.convert()

# Simpan model
with open('model/aplikasi/model.tflite', 'wb') as f:
  f.write(tflite_model)